# Visualizar terremotos

Estos datos sobre terremotos son cortesía de [USGS](https://earthquake.usgs.gov).

{query_address}

Escribe una dirección para ordenar los terremotos recientes por distancia.

In [2]:
target_folder = '/tmp'
query_address = 'David, Chiriquí'

In [3]:
from geopy.geocoders import GoogleV3
g = GoogleV3()
query_location = g.geocode(query_address)
query_point = query_location.point
query_point

Point(8.4007278, -82.4427769, 0.0)

In [4]:
import pandas as pd

In [5]:
table_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.csv'
table = pd.read_csv(table_url, parse_dates=['time'])
table.ix[0]

time                          2017-03-05 22:47:53.310000
latitude                                          -5.999
longitude                                        149.353
depth                                              31.07
mag                                                  6.3
magType                                              mww
nst                                                  NaN
gap                                                   17
dmin                                               3.326
rms                                                 0.99
net                                                   us
id                                            us1000876f
updated                         2017-03-06T16:07:31.040Z
place              32km NW of Kandrian, Papua New Guinea
type                                          earthquake
horizontalError                                      6.6
depthError                                           3.2
magError                       

In [6]:
table = table[['time', 'latitude', 'longitude', 'depth', 'mag', 'place']]
table.ix[0]

time                    2017-03-05 22:47:53.310000
latitude                                    -5.999
longitude                                  149.353
depth                                        31.07
mag                                            6.3
place        32km NW of Kandrian, Papua New Guinea
Name: 0, dtype: object

In [7]:
from geopy.distance import vincenty as get_distance

def get_distance_in_km(x):
    return pd.Series(get_distance(query_point, (x.latitude, x.longitude)).kilometers)
    
table['distanceInKm'] = table.apply(get_distance_in_km, axis=1)
table = table.sort_values(['distanceInKm'])
table[:3]

,time,latitude,longitude,depth,mag,place,distanceInKm
10,2017-02-06 13:02:45.140,3.4493,-74.6715,37.95,5.5,"16km ENE of Colombia, Colombia",1019.708963
4,2017-02-21 14:09:04.410,-19.2839,-63.8990,597.91,6.5,"42km E of Padilla, Bolivia",3676.233725
5,2017-02-18 12:10:17.970,-23.8613,-66.6592,222.00,6.3,"52km NW of San Antonio de los Cobres, Argentina",3960.251187


In [8]:
from datetime import datetime
now = datetime.utcnow()

def get_elapsed_time_in_negative_seconds(x):
    return -1 * (now - x).seconds

geotable = table.copy()
geotable['fillReds'] = geotable['time'].apply(get_elapsed_time_in_negative_seconds)
geotable['radiusInPixelsRange3-27'] = geotable['mag']
geotable.ix[0]

time                                  2017-03-05 22:47:53.310000
latitude                                                  -5.999
longitude                                                149.353
depth                                                      31.07
mag                                                          6.3
place                      32km NW of Kandrian, Papua New Guinea
distanceInKm                                             14313.7
fillReds                                                  -52664
radiusInPixelsRange3-27                                      6.3
Name: 0, dtype: object

In [9]:
from os.path import join

table_path = join(target_folder, 'terremotos.csv')
table.to_csv(table_path, index=False)
print('terremotos_table_path = %s' % table_path)

geotable_path = join(target_folder, 'terremotos-mapa.csv')
geotable.to_csv(geotable_path, index=False)
print('terremotos_satellite_geotable_path = %s' % geotable_path)

terremotos_table_path = /tmp/terremotos.csv
terremotos_satellite_geotable_path = /tmp/terremotos-mapa.csv


# Terremotos visualizados

{terremotos_table}

{terremotos_satellite_geotable}

El radio del círculo indica la magnitud.

El color del círculo indica el tiempo.

[USGS](https://earthquake.usgs.gov) ofrece más tablas sobre terremotos recientes.